Task 1

• Modify the algorithm so that substitution operation cost depends on the
key proximity on QWERTY keyboard.

In [24]:
def load_dictionary(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file]

In [25]:
keyboard_layout = {
    'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3), 't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
    'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3), 'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8),
    'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3), 'b': (2, 4), 'n': (2, 5), 'm': (2, 6)
}

In [30]:
proximity_matrix = {}
for key1, coord1 in keyboard_layout.items():
    proximity_matrix[key1] = {}
    for key2, coord2 in keyboard_layout.items():
        proximity_matrix[key1][key2] = ((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2) ** 0.5



In [27]:
def wagner_fischer(s1, s2):
    len_s1, len_s2 = len(s1), len(s2)
    if len_s1 > len_s2:
        s1, s2 = s2, s1
        len_s1, len_s2 = len_s2, len_s1

    current_row = range(len_s1 + 1)
    for i in range(1, len_s2 + 1):
        previous_row, current_row = current_row, [i] + [0] * len_s1
        for j in range(1, len_s1 + 1):
            add, delete, change = previous_row[j] + 1, current_row[j-1] + 1, previous_row[j-1]
            if s1[j-1] != s2[i-1]:
                substitution_cost = proximity_matrix.get(s1[j-1], {}).get(s2[i-1], float('inf'))
                change += substitution_cost
            current_row[j] = min(add, delete, change)

    return current_row[len_s1]

In [28]:
def spell_check(word, dictionary):
    suggestions = []

    for correct_word in dictionary:
        distance = wagner_fischer(word, correct_word)
        suggestions.append((correct_word, distance))

    suggestions.sort(key=lambda x: x[1])
    return suggestions[:10]

In [31]:
dictionary = load_dictionary("words.txt")
misspelled_word = "wrlod"
suggestions = spell_check(misspelled_word, dictionary)
print(f"Top 10 suggestions for '{misspelled_word}':")
for word, distance in suggestions:
    print(f"{word} (Distance: {distance})")

Top 10 suggestions for 'wrlod':
world (Distance: 2)
wood (Distance: 2.0)
rod (Distance: 2)
well (Distance: 3.0)
food (Distance: 3.0)
road (Distance: 3)
word (Distance: 3)
wed (Distance: 3.0)
los (Distance: 3.0)
wrote (Distance: 3.0)


Another modification: include transposition operation, so that you compute
a Damerau-Levenshtein distance

In [33]:
def load_dictionary(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file]

def wagner_fischer(s1, s2):
    len_s1, len_s2 = len(s1), len(s2)
    if len_s1 > len_s2:
        s1, s2 = s2, s1
        len_s1, len_s2 = len_s2, len_s1

    max_dist = len_s1 + 1
    distances = [[0] * (len_s2 + 1) for _ in range(2)]
    for i in range(len_s1 + 1):
        distances[0][i] = i

    for i in range(1, len_s2 + 1):
        distances[i % 2][0] = i
        for j in range(1, len_s1 + 1):
            add, delete, change = distances[(i - 1) % 2][j] + 1, distances[i % 2][j - 1] + 1, distances[(i - 1) % 2][j - 1]
            if s1[j - 1] != s2[i - 1]:
                change += 1
            distances[i % 2][j] = min(add, delete, change)
            if i > 1 and j > 1 and s1[j - 1] == s2[i - 2] and s1[j - 2] == s2[i - 1]:
                distances[i % 2][j] = min(distances[i % 2][j], distances[(i - 2) % 2][j - 2] + 1)

    return distances[len_s2 % 2][len_s1]

def spell_check(word, dictionary):
    suggestions = []

    for correct_word in dictionary:
        distance = wagner_fischer(word, correct_word)
        suggestions.append((correct_word, distance))

    suggestions.sort(key=lambda x: x[1])
    return suggestions[:10]

# Example Usage
dictionary = load_dictionary("words.txt")
misspelled_word = "wrllod"
suggestions = spell_check(misspelled_word, dictionary)
print(f"Top 10 suggestions for '{misspelled_word}':")
for word, distance in suggestions:
    print(f"{word} (Distance: {distance})")

Top 10 suggestions for 'wrllod':
willow (Distance: 2)
will (Distance: 3)
would (Distance: 3)
world (Distance: 3)
well (Distance: 3)
called (Distance: 3)
yellow (Distance: 3)
follow (Distance: 3)
allow (Distance: 3)
blood (Distance: 3)


Task 2

In [67]:
import nltk
from nltk.corpus import gutenberg
nltk.download('gutenberg')
from collections import Counter
import numpy as np

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [62]:
def create_vocabulary(corpus):
    words = [word.lower() for word in corpus if word.isalpha()]  # Ensure words are alphanumeric
    vocabulary = set(words)
    return vocabulary

In [63]:
def calculate_word_counts(document):
    word_counts = Counter(document)
    return word_counts

In [64]:
def map_to_vector(document, vocabulary):
    vector = np.zeros(len(vocabulary))
    word_counts = calculate_word_counts(document)
    for i, word in enumerate(vocabulary):
        vector[i] = word_counts[word]
    return vector

In [65]:
def compute_dot_product(vector1, vector2):
    return np.dot(vector1, vector2)

In [70]:
text = nltk.Text(nltk.corpus.gutenberg.words('melville-moby_dick.txt'))

In [72]:
vocabulary = create_vocabulary(text)

In [74]:
document1 = text[:500]
document2 = text[500:1000]

In [75]:
vector1 = map_to_vector(document1, vocabulary)
vector2 = map_to_vector(document2, vocabulary)


In [76]:
similarity = compute_dot_product(vector1, vector2)

In [77]:
print("Similarity between documents:", similarity)

Similarity between documents: 937.0
